In [2]:
import pandas as pd

In [4]:
df = pd.read_csv('..\data\\new_procedure.csv')
df.head()

,procedure_name
0,thủ tục chuyển nhượng toàn bộ hoặc một phần dự...
1,thủ tục cấp mới chứng chỉ hành nghề môi giới b...
2,thủ tục cấp lại (cấp đổi) chứng chỉ hành nghề ...
3,thủ tục thông báo nhà ở hình thành trong tương...
4,"thủ tục cho thuê, thuê mua nhà ở xã hội thuộc ..."


In [70]:
df_khaisinh = df[df.procedure_name.str.contains('khai tử')]
df_khaisinh

,procedure_name
101,thủ tục đăng ký khai tử có yếu tố nước ngoài
109,thủ tục ghi vào sổ hộ tịch việc hộ tịch khác c...
113,thủ tục đăng ký lại khai tử có yếu tố nước ngoài
656,thủ tục đăng ký khai tử
657,thủ tục đăng ký khai tử lưu động
658,thủ tục đăng ký lại khai tử


In [71]:
tmp = df_khaisinh.replace(to_replace='thủ tục đăng ký ', value="",regex=True)

In [72]:
list_procedure = tmp.values.flatten().tolist()
list_procedure


['khai tử có yếu tố nước ngoài',
 'thủ tục ghi vào sổ hộ tịch việc hộ tịch khác của công dân việt nam đã được giải quyết tại cơ quan có thẩm quyền của nước ngoài (khai sinh; giám hộ; nhận cha, mẹ, con; xác định cha, mẹ, con; nuôi con nuôi; khai tử; thay đổi hộ tịch)',
 'lại khai tử có yếu tố nước ngoài',
 'khai tử',
 'khai tử lưu động',
 'lại khai tử']

In [48]:
def remove_dup(procedure_list:list):
    mylist = list(dict.fromkeys(procedure_list))
    return mylist


In [76]:
def split_verb_phrase(list_procedure:list):
    result_list = ['có yếu tố nước ngoài', 'cho người đã có hồ sơ, giấy tờ cá nhân','lần đầu','lần hai']
    tmp_list = []
    for procedure in list_procedure:
        procedure = procedure.replace(result_list[0], '').strip()
        procedure = procedure.replace(result_list[1], '').strip()
        tmp_list.append(procedure)
        tmp_list = sorted(remove_dup(tmp_list), key=len, reverse=True)
    result_list = tmp_list + result_list
    return(result_list)

In [74]:
split_verb_phrase(list_procedure)

['thủ tục ghi vào sổ hộ tịch việc hộ tịch khác của công dân việt nam đã được giải quyết tại cơ quan có thẩm quyền của nước ngoài (khai sinh; giám hộ; nhận cha, mẹ, con; xác định cha, mẹ, con; nuôi con nuôi; khai tử; thay đổi hộ tịch)',
 'khai tử lưu động',
 'lại khai tử',
 'khai tử',
 'có yếu tố nước ngoài',
 'cho người đã có hồ sơ, giấy tờ cá nhân']

In [75]:
with open('tmp.txt','w', encoding='utf-8') as f:
    for val in split_verb_phrase(list_procedure):
        f.write("'"+str(val)+"':" + " ['"+str(val)+"'],")
        f.write('\n')

In [97]:
keyword_dictionary = {
    'khai sinh lưu động': ['khai sinh lưu động'],
    'lại khai sinh': ['lại khai sinh'],
    'khai sinh': ['khai sinh'],
    'nhận cha, mẹ, con': ['nhận cha, mẹ, con'],
    'kết hôn lưu động': ['kết hôn lưu động'],
    'lại kết hôn': ['lại kết hôn'],
    'kết hôn': ['kết hôn'],
    'khai tử lưu động': ['khai tử lưu động'],
    'lại khai tử': ['lại khai tử'],
    'khai tử': ['khai tử'],
    'ly hôn': ['ly hôn'],
    'biến động quyền sử dụng đất': ['biến động quyền sử dụng đất'],
    'đăng ký đất đai': ['đăng ký đất đai'],
    'tranh chấp đất đai': ['tranh chấp đất đai'],
    'khám chữa bệnh': ['khám chữa bệnh'],
    'bảo hiểm y tế': ['bảo hiểm y tế'],
    'bảo hiểm xã hội': ['bảo hiểm xã hội'],
    'an toàn lao động': ['an toàn lao động'],
    'trợ cấp thất nghiệp': ['trợ cấp thất nghiệp'],
    'thất nghiệp': ['thất nghiệp'],
    'chứng thực di chúc': ['chứng thực di chúc'],
    'di chúc': ['di chúc'],

    'có yếu tố nước ngoài': ['có yếu tố nước ngoài', 'người nước ngoài', 'nước ngoài'],
    'cho người đã có hồ sơ, giấy tờ cá nhân': ['cho người đã có hồ sơ, giấy tờ cá nhân'],
    'lần đầu': ['lần đầu'],
    'lần hai': ['lần hai'],
    'thanh toán': ['thanh toán'],
    'cấp thẻ': ['cấp thẻ'],
    'một lần': ['một lần'],
    'trợ cấp': ['trợ cấp'],
    'chứng thực': ['chứng thực']

}

In [79]:
import json

In [98]:
with open("../json/keyword.json", "w", encoding='utf-8') as outfile:
    json.dump(keyword_dictionary, outfile, indent=4, ensure_ascii=False)


In [89]:
BOT_MEMORY = {'keywords' : [], 'action':None}

In [95]:
def bot_understand(user_question:str):
    global BOT_MEMORY
    keyword_list = []
    #load keyword dictionary
    with open('../json/keyword.json','r', encoding='utf-8') as f:
        keyword_dict = json.load(f)
    for key in keyword_dict.keys():
        for val in keyword_dict[key]:
            if val in user_question:
                keyword_list.append(key)
                user_question = user_question.replace(val, '')
    BOT_MEMORY.update({'keywords': keyword_list})
    return BOT_MEMORY

In [100]:
bot_understand('Các bước đăng ký lại kết hôn với người nước ngoài')


{'keywords': ['lại kết hôn', 'có yếu tố nước ngoài'], 'action': None}

In [ ]:
# tiếp theo thử nghiệm search
# 1. cộng động từ
# 2. duplicate(df[v1] + df[v2])
# lấy top 5